In [3]:
import pandas as pd
import datetime
import PyPDF2
import numpy as np

In [4]:
new_df = pd.read_csv('test.csv',index_col=0)
# add_df = {
#     "Products": ["3_PersonalLoan","2_BusinessLoan","5_AutoLoan"],
#     "Loan Institution":["OHMY Loan Private Limited","Bank123","Bank123"],
#     "date_opened": [datetime.date(2019, 1, 1),datetime.date(2019, 1, 2),datetime.date(2019, 1, 3)],
#     "Sanction/Credit Limit":[10000,12000,15000],
#     "Balance":[100000,200000,300000],
#     "EMI":[1000,2000,3000],
#     "Paid Principle": [21887,21993,22100],
# }
# new_df = new_df.append(add_df, ignore_index=True)
# new_df = pd.concat([new_df,pd.DataFrame(add_df)])
add_df = {
    "Products": ["3_HousingLoan"],
    "Loan Institution":["OHMY Loan Private Limited"],
    "date_opened": [datetime.date(2019, 1, 1)],
    "Sanction/Credit Limit":[10000],
    "Balance":[100000],
    "EMI":[1000],
    "Paid Principle": [21887],
}
recommendation_string = ""
# new_df = new_df.append(add_df, ignore_index=True)
new_df = pd.concat([new_df,pd.DataFrame(add_df)])
new_df =new_df.reset_index()
new_df

,index,Loan Institution,Sanction/Credit Limit,Balance,EMI,Paid Principle,Delinquencies,Products,date_opened
0,3_PersonalLoan,State Bank Of India,961000,962718,19317,0,True,NaN,NaN
1,3_PersonalLoan,State Bank Of India,1,-813858,0,813859,False,NaN,NaN
2,4_Others,Punjab National Bank,800000,805163,0,0,False,NaN,NaN
3,4_Others,Punjab National Bank,80,80,0,0,False,NaN,NaN
4,4_Others,Punjab National Bank,1000000,1039824,9525,0,True,NaN,NaN
5,4_Others,Uttar Bihar Gramin Bank,300000,319508,0,0,False,NaN,NaN
6,4_Others,Central Bank of India,45000,15145,1058,29855,True,NaN,NaN
7,Total,NaN,3106081,2328580,29900,843714,NaN,NaN,NaN
8,0,OHMY Loan Private Limited,10000,100000,1000,21887,NaN,2_BusinessLoan,2019-01-01


In [5]:
new_df.sort_values(by=['Paid Principle'],ascending=False,inplace=True)
new_df.shape

(9, 9)

In [6]:
top_up_df = new_df.loc[new_df['Products'].isin(['3_PersonalLoan','5_AutoLoan','6_HouseLoan'])]
# new_df.drop(top_up_df.index)
top_up_df = top_up_df.drop_duplicates(subset=['Loan Institution'],keep='first')
new_df = pd.concat([new_df,top_up_df])
new_df

,index,Loan Institution,Sanction/Credit Limit,Balance,EMI,Paid Principle,Delinquencies,Products,date_opened
7,Total,NaN,3106081,2328580,29900,843714,NaN,NaN,NaN
1,3_PersonalLoan,State Bank Of India,1,-813858,0,813859,False,NaN,NaN
6,4_Others,Central Bank of India,45000,15145,1058,29855,True,NaN,NaN
8,0,OHMY Loan Private Limited,10000,100000,1000,21887,NaN,2_BusinessLoan,2019-01-01
0,3_PersonalLoan,State Bank Of India,961000,962718,19317,0,True,NaN,NaN
2,4_Others,Punjab National Bank,800000,805163,0,0,False,NaN,NaN
3,4_Others,Punjab National Bank,80,80,0,0,False,NaN,NaN
4,4_Others,Punjab National Bank,1000000,1039824,9525,0,True,NaN,NaN
5,4_Others,Uttar Bihar Gramin Bank,300000,319508,0,0,False,NaN,NaN


In [ ]:
# Recommendation above 650